In [7]:
import pickle
import numpy as np
from make_BIDS_behav_dict import make_BIDS_behav_dict
import glob
import pandas as pd

In [5]:
story_lens = pickle.load(open('Preprocessing/pickle/story_lens36.pickle', 'rb'))
listen_order = pickle.load(open('Preprocessing/pickle/listen_order36.pickle', 'rb'))
timing = pickle.load(open('Preprocessing/pickle/timing36.pickle', 'rb'))

In [6]:
subs = ['sub-01', 'sub-02', 'sub-03', 'sub-04', 'sub-05', 'sub-06', 'sub-07', 'sub-08', 'sub-09', 'sub-10',
       'sub-12', 'sub-13','sub-14', 'sub-16', 'sub-17', 'sub-18', 'sub-19', 'sub-20', 'sub-21', 'sub-22',
        'sub-23', 'sub-24', 'sub-25', 'sub-26', 'sub-27', 'sub-28', 'sub-29', 'sub-30', 'sub-31', 'sub-32', 'sub-33', 
       'sub-34', 'sub-35', 'sub-36', 'sub-37', 'sub-38']

behav_files = make_BIDS_behav_dict(subs)

['sub-01', 'sub-02', 'sub-03', 'sub-04', 'sub-05', 'sub-06', 'sub-07', 'sub-08', 'sub-09', 'sub-10', 'sub-12', 'sub-13', 'sub-14', 'sub-16', 'sub-17', 'sub-18', 'sub-19', 'sub-20', 'sub-21', 'sub-22', 'sub-23', 'sub-24', 'sub-25', 'sub-26', 'sub-27', 'sub-28', 'sub-29', 'sub-30', 'sub-31', 'sub-32', 'sub-33', 'sub-34', 'sub-35', 'sub-36', 'sub-37', 'sub-38']


In [9]:
stories = [11,12,13,14,21,22,23,24,31,32,33,34,41,42,43,44]

#import stories files
directory_stories = '/data/topdown/stim/story_csv'

filenames_stories = glob.glob(directory_stories + '/*.csv')

dfs_stories = []
    
for filename in filenames_stories:
    dfs_stories.append(pd.read_csv(filename))

# organize in dict   
story_answers = dict()

#read in the location and social event values from the story files 
for s in range(16):
    this_story = int(dfs_stories[s]['story'].iloc[0])
    keys2 = dfs_stories[s]['locationAnswer'].values
    keys3 = dfs_stories[s]['socialAnswer'].values
    story_answers[this_story] = keys2[:, np.newaxis]
    story_answers[this_story] = np.concatenate((story_answers[this_story], keys3[:, np.newaxis]), axis = 1)

In [12]:
answer_TRs = {}

for story in stories:
    answer_TRs[story] = {'location': {}, 'social':{}}

In [40]:
for story in stories:
    for schema in ['location', 'social']:
        if schema == 'location':
            answer_sents = np.where(~np.isnan(story_answers[story][:,0]))
        else:
            answer_sents = np.where(~np.isnan(story_answers[story][:,1]))
        
        start_times = np.zeros((4,0))
        # iterate through all subjects in listen_order and find instances where the current story is listenened to
        for sub in listen_order.keys():
            for run in listen_order[sub].keys():
                this_pd = pd.read_csv(behav_files[sub][run])
                for i in range(listen_order[sub][run].shape[0]):
                    # If the story was listened to by this person in this run...
                    if listen_order[sub][run][i][1] == story:
                        this_count = listen_order[sub][run][i][0]    
                        # find the start times of all the audio clips of that story
                        starts = this_pd['clip_StartTime'].values[this_pd['count']==this_count]
                        starts = starts[~np.isnan(starts)]
                        # Get clip start times of answers
                        st_answer_starts = starts[answer_sents]
                        # see how far all events are from the start of the first sentence
                        this_subtr = starts[0]
                        grounded_event_starts = st_answer_starts - this_subtr
                        # reshape
                        grounded_event_starts = grounded_event_starts.reshape(grounded_event_starts.shape[0], -1)
                        # add the subtracted event starts to start_times
                        start_times = np.hstack((start_times, grounded_event_starts))
        # average the answer times across participants
        avg_st = np.mean(start_times, axis = 1)
        # round up for the TR event length
        for i in range(0,4):
            this_start, this_dec = str(avg_st[i]).split('.')
            this_start = int(this_start) + 1
            avg_st[i] = int(this_start)
        
        answer_TRs[story][schema] = avg_st.astype('int')

In [41]:
answer_TRs

{11: {'location': array([ 17, 113, 125, 203]),
  'social': array([ 13,  75, 133, 187])},
 12: {'location': array([ 10,  44,  93, 171]),
  'social': array([ 18,  74, 128, 190])},
 13: {'location': array([ 27,  37, 113, 151]),
  'social': array([ 12,  84, 131, 193])},
 14: {'location': array([ 18,  46,  88, 150]),
  'social': array([  7,  55, 116, 193])},
 21: {'location': array([ 22,  54,  91, 139]),
  'social': array([  1,  77, 159, 206])},
 22: {'location': array([ 20,  85, 206, 230]),
  'social': array([ 14,  60, 137, 218])},
 23: {'location': array([ 11,  69, 100, 214]),
  'social': array([ 41,  84, 152, 205])},
 24: {'location': array([ 17,  59, 104, 214]),
  'social': array([ 29,  90, 168, 201])},
 31: {'location': array([ 32,  62, 122, 213]),
  'social': array([ 14,  72, 176, 237])},
 32: {'location': array([ 12, 122, 142, 246]),
  'social': array([ 32, 114, 185, 203])},
 33: {'location': array([ 10, 143, 178, 246]),
  'social': array([ 31, 124, 218, 269])},
 34: {'location': arr

In [42]:
with open("target_TRs.pickle", 'wb') as handle:
    pickle.dump(answer_TRs, handle)